In [1]:
import os
import json
import numpy as np

In [2]:
POSTGRES_USER = os.environ.get('POSTGRES_USER', 'admin')
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD', 'admin')
POSTGRES_HOST = os.environ.get('POSTGRES_HOST', 'qualichain.epu.ntua.gr')
POSTGRES_PORT = os.environ.get('POSTGRES_PORT', 5432)
POSTGRES_DB = os.environ.get('POSTGRES_DB', 'api_db')

ENGINE_STRING = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(
    POSTGRES_USER,
    POSTGRES_PASSWORD,
    POSTGRES_HOST,
    POSTGRES_PORT,
    POSTGRES_DB
)

JOB_POSTS_TABLE = 'job_post'


In [3]:
import re

import pandas as pd
from sqlalchemy import create_engine


class JobPostSkillExtractor(object):
    """This Class is used to take job posts and feed them to Dobie"""

    def __init__(self):
        self.engine = create_engine(ENGINE_STRING)
        self.index = 0

    def get_job_posts(self, ids=[]):
        """
        This function is used to get job_posts table from DB

        :return: job_posts table
        """
        if ids:
            tuple_ids = tuple(ids)
            select_query = 'SELECT requirements from "{}" WHERE id in {}'.format(JOB_POSTS_TABLE, tuple_ids)
        else:
            select_query = 'SELECT requirements from {}'.format(JOB_POSTS_TABLE)

        job_posts = pd.read_sql_query(select_query, self.engine)
        return job_posts

    @staticmethod
    def remove_stop_words(job_requirements):
        """
        This function is used to remove stop words from job post requirements
        description

        :param job_requirements: job requirements text
        :return: job requirements without stopwords
        """
        split_txt = job_requirements.split()
        removed_txt = [word for word in split_txt if word not in STOP_WORDS]
        return " ".join(removed_txt)

    def process_job_requirements(self, job_posts_fraction):
        """
        This function receives a job posts data frame, which is a part of the original table
        and preprocess the stored job requirements

        :param job_posts_fraction: pandas data frame
        :return: processed requirements
        """
        # raw_requirements = job_posts_fraction['requirements'].map(
        #     lambda requirement: re.sub(r"[^a-zA-Z0-9]+", ' ', requirement)
        # )
        raw_requirements = job_posts_fraction['requirements']
        removed_stop_words = raw_requirements.map(self.remove_stop_words)
        stipped_from_whitespaces = removed_stop_words.map(
            lambda requirement: requirement.strip()
        )

        processed_requirements = " ".join(stipped_from_whitespaces)
        return processed_requirements


In [4]:
with open('data/skills.json') as json_file:
    skills = json.load(json_file)
    # for p in skills:
    #     print(p)


extractor = JobPostSkillExtractor()
job_posts = extractor.get_job_posts()

In [5]:
skills_df = pd.DataFrame(skills)


In [6]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/vague-qc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
text = job_posts.requirements.iloc[0].lower()

In [8]:
tokenizer = RegexpTokenizer(r'\w+')

tokens = tokenizer.tokenize(text)

In [9]:
def jp_tokenize(text):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens =[]
    for w in tokens:
        if w not in stop_words: 
            filtered_tokens.append(w)
    
    return filtered_tokens

In [10]:
stop_words = set(stopwords.words('english'))

filtered_tokens =[]

for w in tokens:

    if w not in stop_words: 
        filtered_tokens.append(w)
        
filtered_tokens

['manage',
 'implement',
 'data',
 'science',
 'data',
 'analytics',
 'projects',
 'analyze',
 'recovery',
 'business',
 'units',
 'information',
 'needs',
 'order',
 'produce',
 'regular',
 'specialized',
 'reports',
 'design',
 'develop',
 'implement',
 'internal',
 'reporting',
 'processes',
 'conduct',
 'portfolio',
 'segmentation',
 'achieve',
 'optimal',
 'monitoring',
 'draw',
 'conclusions',
 'regarding',
 'efficiency',
 'portfolio',
 'management',
 'proceed',
 'collateral',
 'analysis',
 'focused',
 'detailed',
 'portfolio',
 'mapping',
 'manage',
 'evaluation',
 'analysis',
 'restructuring',
 'plans',
 'rearrangement',
 'methods',
 'engage',
 'sector',
 'analysis',
 'processing',
 'emphasizing',
 'particularities',
 'specific',
 'sectors',
 'actively',
 'involved',
 'npe',
 'forecasting',
 'model',
 'development',
 'implementation',
 'achieve',
 'operational',
 'targets',
 'estimation',
 'monitoring',
 'postgraduate',
 'degree',
 'quantitative',
 'discipline',
 'statistics',


In [11]:
bigrams = nltk.bigrams(filtered_tokens)

In [12]:
def camel_case_split(str): 
    return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str) 

In [13]:
 A_nltk = np.zeros((len(job_posts), len(skills)))

idx = 0
for s in skills:
    skl = s['string']
#     idx_arr = pd.Index(job_posts.requirements).str.contains(skl, regex=False)
#     jp_index = 0
    for jp_idx in range(len(job_posts)):
        jp_reqs = job_posts.requirements.iloc[jp_idx] 
        filtered_tokens = jp_tokenize(jp_reqs)
        for t in filtered_tokens:
            if skl.lower()==t.lower():
                A_nltk[jp_idx][idx] = 1
                break
        if A_nltk[jp_idx][idx] == 0:
            bigrams = nltk.bigrams(filtered_tokens)
            skls = camel_case_split(skl)
            if len(skls)>1:
                bigram = skls[0]+' ' +skls[1]
                for b in bigrams:
                    b_str = b[0]+ b[1]
                    if bigram.lower()==b_str.lower():
                        A_nltk[jp_idx][idx] = 1
                        break
    idx+=1


print(A_nltk)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [14]:
A_nltk.shape

(1777, 95)

In [15]:
skills_to_recommend = skills_df[skills_df['kind']!='topic']

In [16]:
skills_to_recommend

,string,frequencyOfMention,kind
23,GO,2,product
24,IIS,1,product
25,Android,7,product
26,Cassandra,1,product
27,Unix,1,product
28,Linux,3,product
29,Windows,1,product
30,Docker,4,product
31,MsOffice,1,product
32,Debian,1,product


In [17]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

In [18]:
 A_nltk = np.zeros((len(job_posts), len(skills_to_recommend)))

idx = 0
for s in skills_to_recommend.iterrows():
    skl = s[1]['string']
#     idx_arr = pd.Index(job_posts.requirements).str.contains(skl, regex=False)
#     jp_index = 0
    for jp_idx in range(len(job_posts)):
        jp_reqs = job_posts.requirements.iloc[jp_idx] 
        filtered_tokens = jp_tokenize(jp_reqs)
        for t in filtered_tokens:
            if skl.lower()==t.lower():
                A_nltk[jp_idx][idx] = 1
                break
        if A_nltk[jp_idx][idx] == 0:
            bigrams = nltk.bigrams(filtered_tokens)
            skls = camel_case_split(skl)
            if len(skls)>1:
                bigram = skls[0]+' ' +skls[1]
                for b in bigrams:
                    b_str = b[0]+ b[1]
                    if bigram.lower()==b_str.lower():
                        A_nltk[jp_idx][idx] = 1
                        break
    idx+=1


print(A_nltk)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
skills_to_recommend['occurences'] = np.sum(A_nltk, axis=0)

/home/vague-qc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
skills_to_recommend

,string,frequencyOfMention,kind,occurences
23,GO,2,product,85.0
24,IIS,1,product,21.0
25,Android,7,product,91.0
26,Cassandra,1,product,66.0
27,Unix,1,product,67.0
28,Linux,3,product,231.0
29,Windows,1,product,91.0
30,Docker,4,product,138.0
31,MsOffice,1,product,0.0
32,Debian,1,product,12.0


In [21]:
frequent_items = apriori(pd.DataFrame(A_nltk), min_support=0.04, use_colnames=False, max_len=None, verbose=0, low_memory=False)

In [22]:
frequent_items

,support,itemsets
0,0.047833,(0)
1,0.051210,(2)
2,0.129994,(5)
3,0.051210,(6)
4,0.077659,(7)
...,...,...
57,0.078222,"(34, 50)"
58,0.046708,"(41, 35)"
59,0.046145,"(40, 41)"
60,0.042206,"(33, 19, 35)"


In [23]:
rules = association_rules(frequent_items, metric="confidence", min_threshold=0.2)
sorted_rules = rules.sort_values(['confidence'],ascending=False)
sorted_rules.shape

(61, 9)

In [24]:
sorted_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
47,(40),(41),0.047833,0.089477,0.046145,0.964706,10.781650,0.041865,25.798162
56,"(41, 35)",(19),0.046708,0.165447,0.045020,0.963855,5.825752,0.037292,23.089289
49,"(33, 19)",(35),0.048396,0.100732,0.042206,0.872093,8.657594,0.037331,7.030644
40,(36),(34),0.059088,0.235791,0.050647,0.857143,3.635186,0.036715,5.349465
20,(35),(19),0.100732,0.165447,0.082724,0.821229,4.963687,0.066058,4.668279
...,...,...,...,...,...,...,...,...,...
9,(15),(19),0.175014,0.165447,0.044457,0.254019,1.535348,0.015501,1.118732
15,(19),(27),0.165447,0.123241,0.040518,0.244898,1.987140,0.020128,1.161113
29,(34),(24),0.235791,0.120990,0.052335,0.221957,1.834501,0.023807,1.129770
39,(34),(36),0.235791,0.059088,0.050647,0.214797,3.635186,0.036715,1.198304


In [25]:
# skills_to_recommend.iloc[24]

for r in sorted_rules.iterrows():
    antecedents = list(r[1][0])
    consequents = list(r[1][1])
    
    for a in antecedents:
        print(skills_to_recommend['string'].iloc[a])
    
    print('-->')
    
    for c in consequents:
        print(skills_to_recommend['string'].iloc[c])
    
    print("_____________")

CSS3
-->
HTML5
_____________
HTML5
CSS
-->
JavaScript
_____________
HTML
JavaScript
-->
CSS
_____________
Jenkins
-->
Git
_____________
CSS
-->
JavaScript
_____________
Android
-->
IOS
_____________
Docker
-->
Git
_____________
HTML5
-->
JavaScript
_____________
HTML
CSS
-->
JavaScript
_____________
JSON
-->
REST
_____________
HTML
-->
CSS
_____________
HTML5
JavaScript
-->
CSS
_____________
JSON
-->
Git
_____________
Akka
-->
UX
_____________
UX
-->
Akka
_____________
REST
-->
Git
_____________
NoSQL
-->
Git
_____________
NoSQL
-->
Python
_____________
CSS
-->
HTML
_____________
HTML
-->
JavaScript
_____________
JavaScript
CSS
-->
HTML5
_____________
HTML5
-->
CSS
_____________
IOS
-->
Android
_____________
HTML5
-->
CSS3
_____________
CSS
JavaScript
-->
HTML
_____________
HTML5
-->
JavaScript
CSS
_____________
JavaScript
-->
CSS
_____________
HTML
-->
CSS
JavaScript
_____________
CSS
-->
HTML5
_____________
Python
-->
C
_____________
HTML5
-->
Git
_____________
Java
-->
SQL
_________

### Find missing skills from the curriculum

In [28]:
!pip install sqldf

In [29]:
import sqldf

In [31]:
top30_skills = sqldf.run('select * from skills_to_recommend order by occurences desc limit 30')

In [66]:
top30_skill_names = list(top30_skills['string'])
top30_skill_names_lower = [s.lower() for s in top30_skill_names]
top30_skill_names_lower_tuple = tuple(top30_skill_names_lower)



In [96]:
strn = ""
for s in top30_skill_names_lower:
    strn+='(\''+str(s)+'\'),'
strn = strn[:-1]

In [97]:
strn

"('git'),('c'),('sql'),('javascript'),('linux'),('java'),('python'),('rest'),('css'),('html5'),('html'),('ios'),('ux'),('docker'),('akka'),('scripting'),('json'),('php'),('nosql'),('angular'),('api'),('jenkins'),('android'),('windows'),('r'),('react'),('go'),('css3'),('aws'),('tcp')"

In [108]:
query = """VALUES {}
        EXCEPT ALL
            SELECT lower(skill_label)
            FROM   curriculum_designer_skill 
            where skill_type='computer/ data science'""".format(strn)
''
query

"VALUES ('git'),('c'),('sql'),('javascript'),('linux'),('java'),('python'),('rest'),('css'),('html5'),('html'),('ios'),('ux'),('docker'),('akka'),('scripting'),('json'),('php'),('nosql'),('angular'),('api'),('jenkins'),('android'),('windows'),('r'),('react'),('go'),('css3'),('aws'),('tcp')\n        EXCEPT ALL\n            SELECT lower(skill_label)\n            FROM   curriculum_designer_skill \n            where skill_type='computer/ data science'"

In [111]:
from sqlalchemy import create_engine
engine = create_engine(ENGINE_STRING)

def get_table(**kwargs):
    """
    This function is used to load the provided table as a Pandas DataFrame

    :param kwargs: provided kwargs
    :return: pandas DataFrame
    """
    if 'sql_command' in kwargs.keys():
        sql_command = kwargs['sql_command']
        table_df = pd.read_sql_query(sql_command, engine)
    elif 'table' in kwargs.keys():
        table = kwargs['table']
        table_df = pd.read_sql_table(table, engine)
    else:
        table_df = pd.DataFrame()
    return table_df

missing_skills = get_table(sql_command=query)

In [112]:
missing_skills

,column1
0,windows
1,docker
2,r
3,scripting
4,css3
5,api
6,react
7,css
8,json
9,git


In [150]:
skill_dict = {}

for s in missing_skills.iterrows():
    skill = str(s[1]['column1'])

    skill_dict[skill] = []

    for r in sorted_rules.iterrows():
        antecedents = list(r[1][0])
        consequents = list(r[1][1])

        for c in consequents:
            if skill==skills_to_recommend['string'].iloc[c].lower():
                skill_dict[skill].extend(antecedents)



relevant_skills_dict = {}                
for key in skill_dict:
    skill_list = []
    for s in skill_dict[key]:
        skill_list.append(skills_to_recommend['string'].iloc[s].lower())
    
    relevant_skills_dict[key] = list(set(skill_list))
    

relevant_skills_dict

{'windows': [],
 'docker': ['git'],
 'r': [],
 'scripting': [],
 'css3': ['html5'],
 'api': [],
 'react': [],
 'css': ['html', 'html5', 'javascript'],
 'json': ['git', 'rest'],
 'git': ['nosql',
  'rest',
  'json',
  'python',
  'javascript',
  'c',
  'jenkins',
  'css',
  'java',
  'html5',
  'docker'],
 'android': ['ios'],
 'angular': [],
 'tcp': [],
 'rest': ['git', 'json', 'javascript'],
 'ios': ['android'],
 'javascript': ['sql', 'rest', 'git', 'html', 'css', 'java', 'html5'],
 'akka': ['ux'],
 'go': [],
 'html5': ['css3', 'css', 'javascript'],
 'linux': [],
 'aws': [],
 'jenkins': ['git'],
 'php': []}

In [162]:
query = """select cdc.course_title, cdc.id from curriculum_designer_skill
    inner join curriculum_designer_course_course_skills cdccs on curriculum_designer_skill.id = cdccs.skill_id
    inner join curriculum_designer_course cdc on cdc.id = cdccs.course_id
where skill_type='computer/ data science' and lower(skill_label)='{}';"""

courses_for_skill_dict = {}
for key in relevant_skills_dict:
    print(key)
    all_courses_for_skill = []
    for rs in relevant_skills_dict[key]:
        print(rs)
        courses_for_skill = get_table(sql_command=query.format(rs))
        
        if not courses_for_skill.empty:
            print(courses_for_skill)
            all_courses_for_skill.append(courses_for_skill)
            
    courses_for_skill_dict[key] = all_courses_for_skill

windows
docker
git
r
scripting
css3
html5
api
react
css
html
           course_title  id
0             Databases  31
1  Internet Programming  39
html5
javascript
json
git
rest
git
nosql
                          course_title  id
0  Advanced Topics in Database Systems  41
rest
json
python
               course_title  id
0       Distributed Systems  25
1  Programming Languages II  44
2  Digital Communications I  46
javascript
c
                   course_title  id
0             Operating Systems  15
1  Operating Systems Laboratory  19
2       Programming Languages I  32
3     Algorithms and Complexity  33
4       Artificial Intelligence  34
5                     Compilers  37
jenkins
css
java
                  course_title  id
0        Multimedia Technology  18
1  Computer System Performance  21
2          Distributed Systems  25
3      Programming Languages I  32
4         Software Engineering  35
5                    Compilers  37
6         Internet Programming  39
7     Programming Lan

In [163]:
courses_for_skill_dict

{'windows': [],
 'docker': [],
 'r': [],
 'scripting': [],
 'css3': [],
 'api': [],
 'react': [],
 'css': [           course_title  id
  0             Databases  31
  1  Internet Programming  39],
 'json': [],
 'git': [                          course_title  id
  0  Advanced Topics in Database Systems  41,
                 course_title  id
  0       Distributed Systems  25
  1  Programming Languages II  44
  2  Digital Communications I  46,
                     course_title  id
  0             Operating Systems  15
  1  Operating Systems Laboratory  19
  2       Programming Languages I  32
  3     Algorithms and Complexity  33
  4       Artificial Intelligence  34
  5                     Compilers  37,
                    course_title  id
  0        Multimedia Technology  18
  1  Computer System Performance  21
  2          Distributed Systems  25
  3      Programming Languages I  32
  4         Software Engineering  35
  5                    Compilers  37
  6         Internet Programm